<a href="https://colab.research.google.com/github/mohamedshouaib/iti/blob/main/Computer_Vision/Day01/siamesenetwork_faceverification4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mohamedshouaib/iti.git
!cd iti/Computer_Vision/Day01/Siamese

Cloning into 'iti'...
remote: Enumerating objects: 658, done.
remote: Counting objects: 100% (443/443), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 658 (delta 83), reused 0 (delta 0), pack-reused 215 (from 1)
Receiving objects: 100% (658/658), 26.77 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [2]:
import os
import cv2
import csv
import time
import random
import argparse
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as transforms
from datetime import datetime
from pytz import timezone
from tqdm import tqdm

In [3]:
# Hyper Parameters
BATCH_SIZE = 32
IMAGE_SIZE = (150, 150)

In [4]:
# Dataset Loading Functions (Your existing code)
def load_dataset(base_path="iti/Computer_Vision/Day01/Siamese"):
    data = {'train': {}, 'test': {}}

    for person in os.listdir(base_path):
        person_path = os.path.join(base_path, person)
        if not os.path.isdir(person_path):
            continue

        for split in ['Train', 'Test']:
            split_path = os.path.join(person_path, split)
            if not os.path.exists(split_path):
                print(f"Missing {split} folder for {person}")
                continue

            csv_files = [f for f in os.listdir(split_path) if f.endswith('.csv')]
            if not csv_files:
                print(f"No CSV found in {split_path}")
                continue

            csv_path = os.path.join(split_path, csv_files[0])

            genuine = []
            forged = []

            with open(csv_path, 'r') as f:
                try:
                    reader = csv.DictReader(f)
                    row = next(reader)

                    img_col = None
                    label_col = None

                    for col in row.keys():
                        col_lower = col.lower()
                        if 'image' in col_lower or 'name' in col_lower:
                            img_col = col
                        elif 'label' in col_lower or 'class' in col_lower:
                            label_col = col

                    if not img_col or not label_col:
                        raise ValueError("Couldn't detect required columns")

                    f.seek(0)
                    next(reader)

                    for row in reader:
                        img_name = row[img_col].strip()
                        img_path = os.path.join(split_path, img_name)

                        if not os.path.exists(img_path):
                            print(f"Missing image: {img_path}")
                            continue

                        label = row[label_col].strip().lower()
                        if label == 'real' or label == 'genuine':
                            genuine.append(img_path)
                        elif label == 'forged' or label == 'fake':
                            forged.append(img_path)

                except Exception as e:
                    print(f"Error reading {csv_path}: {str(e)}")
                    continue

            if genuine or forged:
                data[split.lower()][person] = {
                    'genuine': genuine,
                    'forged': forged
                }

    return data['train'], data['test']

In [5]:
def generate_triplets(data_dict, split='train'):
    triplets = []
    persons = list(data_dict[split].keys())

    for person in persons:
        genuine = data_dict[split][person]['genuine']
        forged = data_dict[split][person]['forged']

        for i in range(len(genuine)):
            for j in range(i+1, len(genuine)):
                anchor = genuine[i]
                positive = genuine[j]

                for neg in forged:
                    triplets.append((anchor, positive, neg))

    return triplets

In [6]:
# Custom Dataset Class
class SignatureTripletDataset(Dataset):
    def __init__(self, triplets, transform=None):
        self.triplets = triplets
        self.transform = transform or transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        anchor_path, positive_path, negative_path = self.triplets[idx]

        anchor = Image.open(anchor_path).convert('L')
        positive = Image.open(positive_path).convert('L')
        negative = Image.open(negative_path).convert('L')

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

In [7]:
# Model Architecture
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),  # Added dropout

            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),  # Added dropout

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),  # Added dropout

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Dropout(0.5),  # Higher dropout before final layers

            nn.Flatten(),
            nn.Linear(512*18*18, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),  # Dropout in fully connected layer
            nn.Linear(1024, 256)  # Smaller embedding size
        )

    def forward_once(self, x):
        return self.cnn(x)

    def forward(self, input1, input2, input3):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output3 = self.forward_once(input3)
        return output1, output2, output3

In [8]:
# Triplet Loss
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_dist = F.pairwise_distance(anchor, positive, 2)
        neg_dist = F.pairwise_distance(anchor, negative, 2)
        losses = F.relu(pos_dist - neg_dist + self.margin)
        return losses.mean()

In [9]:
def train_model(train_loader, test_loader, args):
    model = SiameseNetwork()
    if args.cuda:
        model = model.cuda()

    # Initialize Triplet Loss criterion with margin from args
    criterion = TripletLoss(margin=args.margin)

    # Optimizer with L2 regularization
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=2, factor=0.5, verbose=True
    )

    # Early stopping setup
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0

    train_losses = []
    val_losses = []

    for epoch in range(args.epochs):
        # Training phase
        model.train()
        epoch_train_loss = 0.0

        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch+1}/{args.epochs} [Train]") as tepoch:
            for anchor, positive, negative in tepoch:
                if args.cuda:
                    anchor, positive, negative = anchor.cuda(), positive.cuda(), negative.cuda()

                optimizer.zero_grad()
                anchor_out, pos_out, neg_out = model(anchor, positive, negative)
                loss = criterion(anchor_out, pos_out, neg_out)

                loss.backward()
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                epoch_train_loss += loss.item()
                tepoch.set_postfix(loss=loss.item())

        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        epoch_val_loss = 0.0

        with torch.no_grad():
            with tqdm(test_loader, unit="batch", desc=f"Epoch {epoch+1}/{args.epochs} [Val]") as vepoch:
                for anchor, positive, negative in vepoch:
                    if args.cuda:
                        anchor, positive, negative = anchor.cuda(), positive.cuda(), negative.cuda()

                    anchor_out, pos_out, neg_out = model(anchor, positive, negative)
                    val_loss = criterion(anchor_out, pos_out, neg_out)
                    epoch_val_loss += val_loss.item()
                    vepoch.set_postfix(loss=val_loss.item())

        avg_val_loss = epoch_val_loss / len(test_loader)
        val_losses.append(avg_val_loss)

        # Update learning rate based on validation loss
        scheduler.step(avg_val_loss)

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered at epoch {epoch+1}")
                break

        # Print epoch summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")
        print(f"Best Val Loss: {best_val_loss:.4f}")

    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()

    return model

In [ ]:
# Main Function
def main():
    # For Jupyter/Colab environment
    class Args:
        epochs = 5
        margin = 1.0
        cuda = torch.cuda.is_available()

    args = Args()

    # Load data
    print("Loading dataset...")
    train_data, test_data = load_dataset()
    print("Generating triplets...")
    train_triplets = generate_triplets({'train': train_data, 'test': test_data}, 'train')
    test_triplets = generate_triplets({'train': train_data, 'test': test_data}, 'test')

    print(f"Training triplets: {len(train_triplets)}")
    print(f"Testing triplets: {len(test_triplets)}")

    # Create datasets
    print("Creating datasets...")
    train_dataset = SignatureTripletDataset(train_triplets)
    test_dataset = SignatureTripletDataset(test_triplets)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Train model
    print("Starting training...")
    model = train_model(train_loader, test_loader, args)

    # Save final model
    torch.save(model.state_dict(), 'final_model.pth')
    print("Training complete. Models saved as 'best_model.pth' and 'final_model.pth'")

if __name__ == '__main__':
    main()

Loading dataset...
Generating triplets...
Training triplets: 19000
Testing triplets: 120
Creating datasets...
Starting training...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/5 [Val]: 100%|██████████| 4/4 [00:01<00:00,  2.67batch/s, loss=0]



Epoch 1 Summary:
Train Loss: 0.0397 | Val Loss: 1.7447
Learning Rate: 1.00e-04
Best Val Loss: 1.7447


Epoch 2/5 [Train]:  39%|███▉      | 234/594 [02:36<04:00,  1.50batch/s, loss=0]

In [ ]:
##################

In [88]:
# Hyper Parameters
BATCH_SIZE = 50

In [89]:
class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, input1, input2, y):
        diff = input1 - input2
        dist_sq = torch.sum(torch.pow(diff, 2), 1)
        dist = torch.sqrt(dist_sq)
        mdist = self.margin - dist
        dist = torch.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
        loss = torch.sum(loss) / 2.0 / input1.size()[0]
        return loss

In [90]:
class LFWDataset(Dataset):

    def __init__(self, root_dir, path_file_dir, transform=None, random_aug=False):
        self.root_dir = root_dir
        path_file = open(path_file_dir, 'r')
        data = []
        for line in path_file:
            line = line.strip()
            img1, img2, label = line.split(' ')
            label = int(label)
            data.append((img1, img2, label))
        self.data = data
        self.transform = transform
        self.random_aug = random_aug
        self.random_aug_prob = 0.7
        path_file.close()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img1, img2, label = self.data[idx]
        img1_file = Image.open(os.path.join(self.root_dir, img1))
        img2_file = Image.open(os.path.join(self.root_dir, img2))
        if self.random_aug:
            img1_file = self.random_augmentation(img1_file, self.random_aug_prob)
            img2_file = self.random_augmentation(img2_file, self.random_aug_prob)

        if self.transform:
            img1_file = self.transform(img1_file)
            img2_file = self.transform(img2_file)
        return (img1_file, img2_file, label)

    def random_augmentation(self, img, prob):
        def rotate(img):
            degree = random.randrange(-30, 30)
            return img.rotate(degree)
        def flip(img):
            return img.transpose(Image.FLIP_LEFT_RIGHT)
        def translate(img):
            d_x = random.randrange(-10, 10)
            d_y = random.randrange(-10, 10)
            img = np.array(img)
            mat = np.float32([[1, 0, d_x], [0, 1, d_y]])
            num_rows, num_cols = img.shape[:2]
            img = cv2.warpAffine(img, mat, (num_cols, num_rows))
            return Image.fromarray(np.uint8(img))
        def scale(img):
            scale = 0.7 + 0.6 * random.random()
            img = np.array(img)
            mat = np.float32([[scale, 0, 0], [0, scale, 0]])
            num_rows, num_cols = img.shape[:2]
            img = cv2.warpAffine(img, mat, (num_cols, num_rows))
            return Image.fromarray(np.uint8(img))

        if random.random() > prob:
            return img

        transform_ops = [rotate, flip, translate, scale]
        op_len = random.randrange(1, len(transform_ops) + 1)
        ops = random.sample(transform_ops, op_len)
        for op in ops:
            img = op(img)
        return img


In [91]:
class Flatten(nn.Module):

    def forward(self, input):
        return input.view(input.size(0), -1)

In [92]:
class SiameseNetwork(nn.Module):

    def __init__(self, contra_loss=False):
        super(SiameseNetwork, self).__init__()

        self.contra_loss = contra_loss

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, padding=2, stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=5, padding=2, stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),

            Flatten(),
            nn.Linear(131072, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(1024)
        )

        self.fc = nn.Sequential(
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        output = self.cnn(x)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        if self.contra_loss:
            return output1, output2
        else:
            output = torch.cat((output1, output2), 1)
            output = self.fc(output)
            return output

In [93]:
def threashold_sigmoid(t):
    """prob > 0.5 --> 1 else 0"""
    threashold = t.clone()
    threashold.data.fill_(0.5)
    return (t > threashold).float()

In [94]:
def threashold_contrastive_loss(input1, input2, m):
    """dist < m --> 1 else 0"""
    diff = input1 - input2
    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    threashold = dist.clone()
    threashold.data.fill_(m)
    return (dist < threashold).float().view(-1, 1)


In [95]:
def cur_time():
    fmt = '%Y-%m-%d %H:%M:%S %Z%z'
    eastern = timezone('US/Eastern')
    naive_dt = datetime.now()
    loc_dt = datetime.now(eastern)
    return loc_dt.strftime(fmt).replace(' ', '_')


In [96]:
def test_against_data(args, label, dataset, siamese_net):
    # Training accuracy
    siamese_net.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0.0
    total = 0.0
    for img1_set, img2_set, labels in dataset:
        labels = labels.view(-1, 1).float()
        if args.cuda:
            img1_set = img1_set.cuda()
            img2_set = img2_set.cuda()
            labels = labels.cuda()
        img1_set = Variable(img1_set)
        img2_set = Variable(img2_set)
        labels = Variable(labels)

        if args.contra_loss:
            output1, output2 = siamese_net(img1_set, img2_set)
            output_labels = threashold_contrastive_loss(output1, output2, args.margin)
        else:
            output_labels_prob = siamese_net(img1_set, img2_set)
            output_labels = threashold_sigmoid(output_labels_prob)

        if args.cuda:
            output_labels = output_labels.cuda()
        total += labels.size(0)
        correct += (output_labels == labels).sum().data[0]

    print('Accuracy of the model on the {} {} images: {} %%'.format(total, label, (100 * correct / total)))


In [102]:
def train(args):
    default_transform = transforms.Compose([
        transforms.Resize(128),  # Changed from Scale to Resize
        transforms.ToTensor(),
    ])
    train_dataset = LFWDataset('./lfw', './train.txt', default_transform, args.randaug)
    print("Loaded {} training data.".format(len(train_dataset)))

    # Data Loader (Input Pipeline)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True)

    siamese_net = SiameseNetwork(args.contra_loss)
    if args.cuda:
        siamese_net = siamese_net.cuda()

    # Loss and Optimizer
    if args.contra_loss:
        criterion = ContrastiveLoss(margin=args.margin)
    else:
        criterion = nn.BCELoss()

    optimizer = torch.optim.Adam(siamese_net.parameters())

    # Train the Model
    num_epochs = args.epoch
    for epoch in range(num_epochs):
        for i, (img1_set, img2_set, labels) in enumerate(train_loader):
            if args.cuda:
                img1_set = img1_set.cuda()
                img2_set = img2_set.cuda()
                labels = labels.cuda()

            img1_set = Variable(img1_set)
            img2_set = Variable(img2_set)
            labels = Variable(labels.view(-1, 1).float())

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            if args.contra_loss:
                output1, output2 = siamese_net(img1_set, img2_set)
                loss = criterion(output1, output2, labels)
            else:
                output_labels_prob = siamese_net(img1_set, img2_set)
                loss = criterion(output_labels_prob, labels)

            loss.backward()
            optimizer.step()

        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, loss.item()))

    # Training accuracy
    test_against_data(args, 'training', train_loader, siamese_net)

    # Save the Trained Model
    model_file_name = "{}_{}".format(cur_time(), args.model_file)
    torch.save(siamese_net.state_dict(), model_file_name)
    print("Saved model at {}".format(model_file_name))
    return siamese_net

In [103]:
def test(args, siamese_net=None):
    if not siamese_net:
        saved_model = torch.load(args.model_file)
        siamese_net = SiameseNetwork(args.contra_loss)
        siamese_net.load_state_dict(saved_model)

    if args.cuda:
        siamese_net = siamese_net.cuda()

    default_transform = transforms.Compose([
        transforms.Scale(128),
        transforms.ToTensor(),
    ])
    test_dataset = LFWDataset('./lfw', './test.txt', default_transform)
    print("Loaded {} test data.".format(len(test_dataset)))

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False)

    test_against_data(args, "testing", test_loader, siamese_net)


In [104]:
def main():
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument("action", nargs='?', choices=['train', 'test', 'train_test'], default='train_test')
    arg_parser.add_argument("model_file", nargs='?', help="model file path", default='siamese.pkl')
    arg_parser.add_argument("-e", "--epoch", type=int, help="training epochs", default=1)
    arg_parser.add_argument("-m", "--margin", type=float, help="training epochs", default=1.0)
    arg_parser.add_argument("-c", "--cuda", action='store_true', default=False)
    arg_parser.add_argument("-r", "--randaug", action='store_true', default=False)
    arg_parser.add_argument("-cl", "--contra_loss", action='store_true', default=False)

    # For Colab/Jupyter, use empty list for args
    args = arg_parser.parse_args([]) if 'google.colab' in str(get_ipython()) else arg_parser.parse_args()

    print("Invoke {} with args {}".format(args.action, args))
    if args.action == "train":
        train(args)
    elif args.action == "test":
        test(args)
    elif args.action == 'train_test':
        siamese_net = train(args)
        test(args, siamese_net)

In [105]:
if __name__ == '__main__':
    class Args:
        action = 'train_test'
        model_file = 'siamese.pkl'
        epoch = 10
        margin = 1.0
        cuda = torch.cuda.is_available()
        randaug = False
        contra_loss = False

    args = Args()
    main()

Invoke train_test with args Namespace(action='train_test', model_file='siamese.pkl', epoch=1, margin=1.0, cuda=False, randaug=False, contra_loss=False)


FileNotFoundError: [Errno 2] No such file or directory: './train.txt'

In [ ]:
def evaluate_model(model, test_triplets):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for triplet in test_triplets:
            a, p, n = load_triplet_images(triplet)
            a = torch.tensor(a).unsqueeze(0).unsqueeze(0).to(device)
            p = torch.tensor(p).unsqueeze(0).unsqueeze(0).to(device)
            n = torch.tensor(n).unsqueeze(0).unsqueeze(0).to(device)

            out_a, out_p = model.forward_once(a), model.forward_once(p)
            out_n = model.forward_once(n)

            d_ap = F.pairwise_distance(out_a, out_p)
            d_an = F.pairwise_distance(out_a, out_n)

            if d_ap.item() < d_an.item():
                correct += 1
            total += 1

    print(f"Accuracy on test set: {correct / total:.2f}")


In [ ]:
# Evaluate model on the test triplets
evaluate_model(model, test_triplets)

In [ ]:
torch.save(model.state_dict(), 'siamese_signature_model.pth')

# To load later
# model.load_state_dict(torch.load('siamese_signature_model.pth'))
# model.eval()


In [ ]:
def test_signature_pair(img_path1, img_path2, model, threshold=1.5):
    model.eval()
    img1, img2 = load_triplet_images((img_path1, img_path2, img_path2))[:2]  # ignore negative

    img1 = torch.tensor(img1).unsqueeze(0).unsqueeze(0).to(device)
    img2 = torch.tensor(img2).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        out1 = model.forward_once(img1)
        out2 = model.forward_once(img2)

    distance = F.pairwise_distance(out1, out2).item()
    print(f"Distance: {distance:.4f}")
    return distance < threshold  # True if similar


In [ ]:
# Example real vs. real (should return True if genuine match)
img_path1 = 'iti/Computer_Vision/Day01/Siamese/personA/Test/personA_29.png'
img_path2 = 'iti/Computer_Vision/Day01/Siamese/personA/Test/personA_13.png'

# Example real vs. forged (should return False if forged)
img_path3 = 'iti/Computer_Vision/Day01/Siamese/personA/Test/personA_29.png'
img_path4 = 'iti/Computer_Vision/Day01/Siamese/personA/Test/personA_33.png'

# Test genuine match
is_match = test_signature_pair(img_path1, img_path2, model)
print("Match (real vs. real):", is_match)

# Test forged case
is_forged = test_signature_pair(img_path3, img_path4, model)
print("Match (real vs. forged):", is_forged)